# Nodejs MNIST model Deployment

 * Wrap a nodejs tensorflow model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
 
## Depenencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * pip install grpcio-tools
 * node (version>=8.11.0)
 * npm

## Train locally using npm commands
 This model takes in mnist images of size 28x28x1 as input and outputs an array of size 10 with prediction of each digits from 0-9

In [ ]:
!make train && make clean_build

Training creates a model.json file and a weights.bin file which is utilized for prediction

## Prediction using REST API on the docker container

In [ ]:
!s2i build . seldonio/seldon-core-s2i-nodejs:0.1 node-s2i-mnist-model:0.1

In [ ]:
!docker run --name "nodejs_mnist_predictor" -d --rm -p 5000:5000 node-s2i-mnist-model:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

Send some random features that conform to the contract

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p -t

In [ ]:
!docker rm nodejs_mnist_predictor --force

## Prediction using GRPC API on the docker container

In [ ]:
!s2i build -E ./.s2i/environment_grpc . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-mnist-model:0.2

In [ ]:
!docker run --name "nodejs_mnist_predictor" -d --rm -p 5000:5000 node-s2i-mnist-model:0.2

In [ ]:
!cd ../../../wrappers/testing && make build_protos

Send some random features that conform to the contract

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p -t --grpc

In [ ]:
!docker rm nodejs_mnist_predictor --force

## Test on Minikube

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-nodejs:0.1 node-s2i-mnist-model:0.1

In [ ]:
!kubectl create -f nodejs_mnist_deployment.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' 

In [ ]:
!cd ../../../util/api_tester && make build_protos 

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete

In [ ]:
!make clean